## ETHUSD rates 
`This notebook is still a work in process`

- The ETHUSD rates will be joined with the transaction data in ETH, then used to convert the token prices in ETH to USD.
- Source: CoinAPI https://docs.coinapi.io/?python#timeseries-data-get
- The free access allows queries of 100k records daily.

In [ ]:
import pandas as pd
import requests

In [ ]:
import os
path = str(os.getcwd()) + '/'

Sample request:

In [ ]:
my_key = 'my_key'

# specify the time frame & interval
url = 'https://rest.coinapi.io/v1/exchangerate/ETH/USD/history?period_id=1MIN&time_start=2021-06-11T00:00:00&time_end=2022-03-30T23:59:00&limit=100000'

headers = {'X-CoinAPI-Key' : my_key}
response = requests.get(url, headers=headers)

# convert response in json to a dataframe
ethusd_df = pd.DataFrame.from_dict(response.json())

In [ ]:
# write to a local file, if needed
# ethusd_df.to_excel('file_name.xlsx', index=False)

As the free API comes with daily limit, I've collected ETHUSD data in separate files. Let's put them together:

In [ ]:
ethusd_2021_sep_oct = pd.read_excel(path + 'Fidenza_updated/ethusd_2021_sep_oct.xlsx')
ethusd_2021_nov_dec = pd.read_excel(path + 'Fidenza_updated/ethusd_2021_nov_dec.xlsx')
ethusd_2022_jan_feb = pd.read_excel(path + 'Fidenza_updated/ethusd_2022_jan_feb.xlsx')
ethusd_2022_mar = pd.read_excel(path + 'Fidenza_updated/ethusd_2022_mar.xlsx')
# data from Jun-Aug 2021 will be added soon

In [ ]:
# a quick glance at the raw data
ethusd_2022_mar.head(2)

In [ ]:
ethusd_2022_mar.info()

Define a function to clean the data first:

In [ ]:
def ethusd_prep(df):
    # take the average of the OHLC rates per minute as our reference
    df['ethusd_rate'] = df[['rate_open', 'rate_high', 'rate_low', 'rate_close']].mean(axis=1)
    
    # fix the datetime format
    df.loc[:,'time_period_start'] = df['time_period_start'].astype(str).str.replace('T', ' ').str[:-9]
    df.loc[:,'time_period_start'] = pd.to_datetime(df['time_period_start'], format='%Y-%m-%d %H:%M:%S')
    
    # drop unwanted columns
    df.drop(columns=['time_period_end', 'time_open', 'time_close',
                     'rate_open', 'rate_high', 'rate_low', 'rate_close', 'Unnamed: 0'], inplace=True)

    # rename column to merge with historical data later
    df.rename(columns={'time_period_start': 'block_time'}, inplace=True)
    
    # sort by descending datetime
    df.sort_values(by='block_time', ascending=False, inplace=True)
    return df

In [ ]:
for df in [ethusd_2022_mar, ethusd_2022_jan_feb, ethusd_2021_nov_dec, ethusd_2021_sep_oct]:
    df = ethusd_prep(df)

Concaternate all relevant datasets for the full historical ETHUSD rate, currently from 2021-06-11 (launch date) to 2022-03-30:

In [ ]:
ethusd_rates_to_20220330 = pd.concat([ethusd_2022_mar, ethusd_2022_jan_feb, 
                                      ethusd_2021_nov_dec, ethusd_2021_sep_oct])

In [ ]:
ethusd_rates_to_20220330.reset_index(drop=True, inplace=True)

***Regular update***

In [ ]:
my_key = ''

# specify the update timeframe & frequency
url = 'https://rest.coinapi.io/v1/exchangerate/ETH/USD/history?period_id=1MIN&time_start=2021-09-01T00:00:00&time_end=2021-10-31T23:59:00&limit=100000'

headers = {'X-CoinAPI-Key' : my_key}
response = requests.get(url, headers=headers)

ethusd_update = pd.DataFrame.from_dict(response.json())

In [ ]:
# clean the new data
ethusd_clean_update = ethusd_prep(df)

# concat to the current table
ethusd_rates_to_new_date = pd.concat([ethusd_clean_update, ethusd_rates_to_20220330])

# reset index
ethusd_rates_to_new_date.reset_index(drop=True, inplace=True)

# update local file 
ethusd_rates_to_new_date.to_csv('file_path/ethusd_rates_to_new_date.csv', index=False)